# Handwritten Digits Image Processing Dataset

## Loading dataset

In [1]:
import numpy as np
def load_data(path):
    with np.load(path) as f:
        x_train, y_train = f['x_train'], f['y_train']
        x_test, y_test = f['x_test'], f['y_test']
        return (x_train, y_train), (x_test, y_test)

(x_train, y_train), (x_test, y_test) = load_data('mnist.npz')

In [2]:
x_train.shape

(60000, 28, 28)

Insights:
- The dataset contains 60,000 small square 28×28 pixel grayscale images of handwritten single digits between 0 and 9

## Business Case:
- The task is to classify a given image of a handwritten digit into one of 10 classes representing integer values from 0 to 9, inclusively.

## Importing Required Libraries

In [3]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

## Scaling input data

In [4]:
#Scaling input data
# Input data is vary from 0(Black) to 255(White). so here scaling in between 0 to 1
num_of_trainImgs = x_train.shape[0] #60000 here
num_of_testImgs = x_test.shape[0] #10000 here
img_width = 28
img_height = 28

x_train = x_train.reshape(x_train.shape[0], img_height, img_width, 1)
x_test = x_test.reshape(x_test.shape[0], img_height, img_width, 1)
input_shape = (img_height, img_width, 1)


x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train / 255
x_test = x_test / 255

## Conversion of class vectors into binary values

In [5]:
#Conversion of class vectors into binary values
num_classes = 10
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)



## Defining model architecture

In [6]:
#defining model architecture

img_width = 28
img_height = 28

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

## Compiling the model

In [7]:
import tensorflow
#Compiling the model
model.compile(loss=keras.losses.categorical_crossentropy,optimizer='adam',metrics=['accuracy'])

## Fitting model to training data

In [8]:
#Fitting model to training data
model.fit(x_train, y_train, epochs=12,steps_per_epoch = 1714)

Epoch 1/12
1714/1714 [==============================] - 166s 94ms/step - loss: 0.1838 - accuracy: 0.9445
Epoch 2/12
1714/1714 [==============================] - 164s 96ms/step - loss: 0.0771 - accuracy: 0.9773
Epoch 3/12
1714/1714 [==============================] - 160s 93ms/step - loss: 0.0594 - accuracy: 0.9821
Epoch 4/12
1714/1714 [==============================] - 160s 93ms/step - loss: 0.0481 - accuracy: 0.9851
Epoch 5/12
1714/1714 [==============================] - 160s 93ms/step - loss: 0.0413 - accuracy: 0.9866
Epoch 6/12
1714/1714 [==============================] - 162s 95ms/step - loss: 0.0357 - accuracy: 0.9889
Epoch 7/12
1714/1714 [==============================] - 161s 94ms/step - loss: 0.0302 - accuracy: 0.9904
Epoch 8/12
1714/1714 [==============================] - 162s 94ms/step - loss: 0.0293 - accuracy: 0.9905
Epoch 9/12
1714/1714 [==============================] - 162s 94ms/step - loss: 0.0254 - accuracy: 0.9918
Epoch 10/12
1714/1714 [==============================] 

## Model evaluation

In [9]:
#model evaluation
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Test loss: 0.030465831980109215
Test accuracy: 0.9926000237464905


## Model saving

In [10]:
#Model saving
model.save('trained_model.h5')

## Digit Recognition

In [11]:
from keras.models import load_model
from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab, Image
import numpy as np

model = load_model('trained_model.h5')

def predict_digit(img):
    #resize image to 28x28 pixels
    img = img.resize((28,28))
    #convert rgb to grayscale
    img = img.convert('L')
    img = np.array(img)
    #reshaping to support our model input and normalizing
    img = img.reshape(1,28,28,1)
    img = img/255.0
    #predicting the class
    res = model.predict([img])[0]
    return np.argmax(res), max(res)

class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)

        self.x = self.y = 0

        # Creating elements
        self.canvas = tk.Canvas(self, width=300, height=300, bg = "white", cursor="cross")
        self.label = tk.Label(self, text="Thinking..", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text = "Recognise", command = self.classify_handwriting) 
        self.button_clear = tk.Button(self, text = "Clear", command = self.clear_all)

        # Grid structure
        self.canvas.grid(row=0, column=0, pady=2, sticky=W, )
        self.label.grid(row=0, column=1,pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)

        #self.canvas.bind("<Motion>", self.start_pos)
        self.canvas.bind("<B1-Motion>", self.draw_lines)

    def clear_all(self):
        self.canvas.delete("all")

    def classify_handwriting(self):
        HWND = self.canvas.winfo_id() # get the handle of the canvas
        rect = win32gui.GetWindowRect(HWND) # get the coordinate of the canvas
        im = ImageGrab.grab(rect)

        digit, acc = predict_digit(im)
        self.label.configure(text= str(digit)+', '+ str(int(acc*100))+'%')

    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r=8
        self.canvas.create_oval(self.x-r, self.y-r, self.x + r, self.y + r, fill='black')

app = App()
mainloop()

1/1 [==============================] - 0s 31ms/step


# Conclusion:

## CNN is having high accuracy and very effective in image recognition problem as compared to ANN and RNN.
### Initially we got dataset from piece of code and then did data preprocessing.
### Then followed CNN process and got best accurate value and less error for test.
### To check model performance we created GUI using Tkinter and got best results

## By above performance we conclude that CNN gives best performance to recognize HANDWRITTEN DIGITS.